In [1]:
import torch
import torch.nn as nn

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from torchsummary import summary

In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch

https://www.youtube.com/watch?v=IKOHHItzukk&ab_channel=deeplizard

In [ ]:
class NNModel(nn.Module):
    def __init__(self):
        super(NNModel,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2,return_indices=True)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2,return_indices=True)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2,return_indices=True)
        )
        
        self.fc1 = nn.Sequential(
            nn.Flatten(start_dim=1),
            nn.Linear(in_features=64,out_features=32),
            nn.ReLU(),
            nn.Dropout(p=0.5),
        )    
        
        self.fc2 = nn.Sequential(
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
        
    def forward(self,x):
        x, indices1 = self.conv1(x)
        x, indices2 = self.conv2(x)
        x, indices3 = self.conv3(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [ ]:
model = NNModel()
model

In [ ]:
summary(model,input_size=(3, 28,28), batch_size=16)

In [ ]:
x = torch.randn(16,3,28,28)
y = model(x)
print("====>",x.shape,y.shape)

In [ ]:
for name,param in model.named_parameters():
    print(name,"\t", param.shape)

In [ ]:
m = nn.ConvTranspose2d(16, 3, 3, stride=2)
x = torch.randn(20, 16, 5, 5)
y = m(x)
x.shape,y.shape

In [ ]:
pool = nn.MaxPool2d(2, 2, return_indices=True)
unpool = nn.MaxUnpool2d(2, 2)

print("x:",x.shape)
out1, indices = pool(x)
print("pool1 x:",out1.shape, "idx:",indices.shape)
out2 = unpool(out1, indices)
print("unpool1 x:",out2.shape)

In [ ]:
x = torch.rand(2, 6, 7, 7)
pool = nn.MaxPool2d(2, 2,padding=1, return_indices=True)
unpool = nn.MaxUnpool2d(2, 2)

out, indices = pool(x)
out = unpool(out, indices)

In [ ]:
self.convT1 = nn.Sequential(
            nn.ConvTranspose2d( in_channels=12, out_channels=6, kernel_size=3),
            nn.ReLU(),
        )
        
        self.unpool1 = nn.MaxUnpool2d(2, stride=2)

In [ ]:
class CatsVsDogsModel(nn.Module):
    def __init__(self):
        super(CatsVsDogsModel, self).__init__()

        self.cnn_model = nn.Sequential(
            nn.Conv2d(3, 16, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        self.fc_model = nn.Sequential(
            nn.Linear(64 * 26 * 26, 64),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.cnn_model(x)
        x = x.view(x.size(0), -1)
        x = self.fc_model(x)
        return x

In [ ]:
c = CatsVsDogsModel()
c.load_state_dict(torch.load("./model_CNN.pt"))
c.eval()

In [102]:
class BaseUNetModel(nn.Module):
    def __init__(self):
        super(BaseUNetModel,self).__init__()
        self.down_conv_layer_1 = nn.Sequential(
            nn.Conv2d(1, 64 , kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
        )        
 
        self.down_conv_layer_2 = nn.Sequential(
            nn.Conv2d(64, 128 , kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.InstanceNorm2d(128)
        ) 
    
        self.down_conv_layer_3 = nn.Sequential(
            nn.Conv2d(128, 256 , kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.InstanceNorm2d(128)
        ) 
        
        self.down_conv_layer_4 = nn.Sequential(
            nn.Conv2d(256, 256 , kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.InstanceNorm2d(128),
            nn.Dropout(0.5),   
        ) 
        
        self.up_conv_layer_1 = nn.Sequential(
            nn.ConvTranspose2d(256, 256, kernel_size=(4,5), stride=2, padding=1),
            nn.InstanceNorm2d(256),
            nn.ReLU(), 
            nn.Dropout(0.5), 
        )
        
        self.up_conv_layer_2 = nn.Sequential(
            nn.ConvTranspose2d(512, 128, kernel_size=4, stride=2, padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(), 
            nn.Dropout(0.5), 
        )
        
        self.up_conv_layer_3 = nn.Sequential(
            nn.ConvTranspose2d(256, 64, kernel_size=(4,5), stride=2, padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU(), 
        )
                
        self.upsample_layer = nn.Upsample(scale_factor=2)
        self.zero_pad = nn.ZeroPad2d((1, 0, 1, 0))
        self.conv_layer_final = nn.Conv2d(128, 1, kernel_size=2,stride=1, padding=0)
        
        
    def forward(self,x):
        print('x', x.shape)
        enc1 = self.down_conv_layer_1(x) # [4, 64, 32, 469]
        print('1', enc1.shape)
        enc2 = self.down_conv_layer_2(enc1) # [4, 128, 16, 234]
        print('2', enc2.shape)
        enc3 = self.down_conv_layer_3(enc2) # [4, 256, 8, 117]
        print('3', enc3.shape)
        enc4 = self.down_conv_layer_4(enc3) # [4, 256, 4, 58]
        print('4', enc4.shape)
        print("only up.....")
        
        dec1 = self.up_conv_layer_1(enc4)
        print('d1', dec1.shape) # [4, 256, 8, 117]
                 
        dec13 = torch.cat((dec1, enc3), 1)
        print('d13', dec13.shape) #[4, 512, 8, 117]
        
        dec2 = self.up_conv_layer_2(dec13)
        print('d2', dec2.shape) # [4, 128, 16, 234]
                 
        dec22 = torch.cat((dec2, enc2), 1)
        print('d22', dec22.shape) #[4, 256, 16, 234]
        
        dec3 = self.up_conv_layer_3(dec22)
        print('d3', dec3.shape) # [4, 256, 16, 234]
                 
        dec31 = torch.cat((dec3, enc1), 1)
        print('d31', dec31.shape) #[4, 256, 16, 234]
        
        final = self.upsample_layer(dec31)
        final = self.zero_pad(final)
        final = self.conv_layer_final(final)

        print(final.shape)
        return final

In [100]:
model = BaseUNetModel()

In [101]:
summary(model,input_size=(1, 64,938), batch_size=64)

x torch.Size([2, 1, 64, 938])
1 torch.Size([2, 64, 32, 469])
2 torch.Size([2, 128, 16, 234])
3 torch.Size([2, 256, 8, 117])
4 torch.Size([2, 256, 4, 58])
only up.....
d1 torch.Size([2, 256, 8, 117])
d13 torch.Size([2, 512, 8, 117])
d2 torch.Size([2, 128, 16, 234])
d22 torch.Size([2, 256, 16, 234])
d3 torch.Size([2, 64, 32, 469])
d31 torch.Size([2, 128, 32, 469])
torch.Size([2, 1, 64, 938])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [64, 64, 32, 469]           1,088
         LeakyReLU-2          [64, 64, 32, 469]               0
            Conv2d-3         [64, 128, 16, 234]         131,200
         LeakyReLU-4         [64, 128, 16, 234]               0
    InstanceNorm2d-5         [64, 128, 16, 234]               0
            Conv2d-6          [64, 256, 8, 117]         524,544
         LeakyReLU-7          [64, 256, 8, 117]               0
    InstanceNorm2d-8          